In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from tqdm import tqdm
import time
import pandas as pd

#크롬 웹드라이버 설치 필수
#코랩으로는 다른 라이브러리로 해야하는 듯함. 지금 코드는 주피터용입니다.

s = Service(r"C:\Users\user\chromedriver.exe")

# 타자 투수 데이터는 주소가 끝에만 바뀌므로 for문 활용
url_base = 'https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx/{category}'
category_list = ['Hitter/Basic1.aspx', 'Pitcher/Basic1.aspx']

driver = webdriver.Chrome(service = s)


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [95]:
hit_data = []


In [83]:
is_pitcher = 0
hit_data = []
category_data_list_hit = []

pitch_data_pitch = []
category_data_list_pitch = []

In [84]:
for category in category_list:
    is_pitcher += 1
    category_data_list = []
    
    url = url_base.format(category = category)
    driver.get(url)
    driver.maximize_window()
    
    # 연도나 이닝처럼 고르는 영역 어떻게 코드 짜여있는지 확인하고 맞는 요소로 대입하여함. 아래는 kbo기록실 타자 연도별 기준임!
    year_list = [x.strip() for x in driver.find_element(By.CLASS_NAME,'select02').text.split('\n') if len(x) >1]
    
    # 처음에서부터 3개년도.
    for year in tqdm(year_list[:3]):
        driver.find_element(By.CLASS_NAME,'select02').click()
        xpath_formatxpath_format = '//option[@value="{year}"]'
        driver.find_element(By.XPATH, xpath_format.format(year = year)).click()
    
        time.sleep(0.1)
        temp_list = [x.split(' ') for x in driver.find_element(By.TAG_NAME, 'table').text.split('\n')]
        temp_data = pd.DataFrame(columns = temp_list[0])
    
    # 하단 try-except문은 투수데이터 가져올때 이닝이 정수로 안떨어지고 분수로 있는 경우를 대비한것. 필요없다면 제외
    # 필요없어도 temp_list안의 데이터는 옮겨줘야함.
    
        for i,temp in enumerate(temp_list[1:-1]):
            try:
                temp_data.loc[i] = temp
                time.sleep(0.5)
            except:
                ip_index = temp_list[0].index('IP')
                ip = temp[ip_index]
                del temp[13]
                temp[13] = ip+' '+temp[13]
                
                temp_data.loc[i] = temp
                time.sleep(0.5)
        temp_data['Year'] = year
        category_data_list.append(temp_data)
        time.sleep(0.1)
        time.sleep(1)
    
    if is_pitcher == 1:
        category_data_list_hit = pd.concat(category_data_list)
    else:
        category_data_list_pitch = pd.concat(category_data_list)



100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:16<00:00,  5.57s/it]


In [97]:
hit_data.append(category_data_list_hit)

In [107]:
hit_data

[  순위   팀명    AVG    G    PA    AB    R     H   2B  3B   HR    TB  RBI  SAC  \
 0  1   롯데  0.280  133  5308  4521  718  1265  247  24  121  1923  671   82   
 1  2  KIA  0.278  133  5189  4542  683  1263  221  17  142  1944  637   51   
 2  3   삼성  0.277  133  5280  4553  739  1262  244  21  162  2034  695   69   
 3  4   두산  0.276  133  5303  4583  732  1266  221  26  130  1929  688   54   
 4  5   LG  0.276  133  5319  4521  669  1247  219  24   85  1769  630  114   
 5  6   한화  0.275  133  5259  4556  659  1255  211  15  148  1940  628   75   
 6  7   현대  0.268  133  5225  4445  711  1192  166  20  169  1905  667   77   
 7  8   SK  0.260  133  5077  4400  596  1143  215  11  113  1719  555  122   
 0  1   삼성  0.284  133  5304  4648  777  1321  266  16  191  2192  731   55   
 1  2   SK  0.270  133  5090  4475  618  1207  204  27  158  1939  587   96   
 2  3  KIA  0.269  133  5200  4575  643  1230  220  14  120  1838  611   45   
 3  4   LG  0.261  133  5121  4492  583  1173  212  